In [2]:
from google.cloud import bigquery
from google import genai
import os

In [3]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE SCHEMA IF NOT EXISTS aurora_bay;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [11]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE TABLE aurora_bay.faqs (
  question STRING,
  answer STRING
);

LOAD DATA OVERWRITE aurora_bay.faqs
FROM FILES (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv'],
  skip_leading_rows = 1
);
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [12]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE MODEL aurora_bay.embedding_model
REMOTE WITH CONNECTION DEFAULT
OPTIONS (ENDPOINT = 'text-embedding-005');
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [15]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
CREATE OR REPLACE TABLE aurora_bay.faqs_embedded AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
  MODEL aurora_bay.embedding_model,
  (SELECT string_field_0, string_field_1, CONCAT(string_field_0, ' ', string_field_1) AS content
   FROM aurora_bay.faqs)
);
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

TableWidget(page_size=10, row_count=1, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes

In [23]:
bq = bigquery.Client()
gemini = genai.Client(vertexai=True, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location="us-central1")

def ask(question):
    results = bq.query("""
        SELECT base.string_field_0 AS question, base.string_field_1 AS answer, distance
        FROM VECTOR_SEARCH(
            TABLE aurora_bay.faqs_embedded, 'ml_generate_embedding_result',
            (SELECT ml_generate_embedding_result
             FROM ML.GENERATE_EMBEDDING(
                 MODEL aurora_bay.embedding_model,
                 (SELECT @q AS content))),
            top_k => 3, distance_type => 'COSINE')
    """, job_config=bigquery.QueryJobConfig(
        query_parameters=[bigquery.ScalarQueryParameter("q", "STRING", question)]
    )).result()

    context = "\n".join([f"Q: {r.question}\nA: {r.answer}" for r in results])

    response = gemini.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"Answer based on these FAQs only:\n\n{context}\n\nQuestion: {question}"
    )
    return response.text

In [24]:
print(ask("What time does the library open?"))
print(ask("Where can I watch whales?"))

The library opens at 9 AM Monday through Friday and at 10 AM on Saturdays.

According to the FAQs, the Aurora Bay Harbor area offers whale watching tours.

